In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

In [5]:
from keras.datasets import boston_housing
(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

In [6]:
train_data.shape

(404, 13)

In [7]:
test_data.shape

(102, 13)

In [10]:
train_targets.shape

(404,)

In [11]:
test_targets.shape

(102,)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(train_data)
X_test = sc.fit_transform(test_data)

In [13]:
X_train

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [14]:
X_test

array([[ 2.8040301 , -0.50784934,  0.96960877, ...,  0.90513041,
        -4.27829517,  2.51324773],
       [-0.55530596, -0.50784934, -0.17801704, ..., -0.28485844,
         0.3909446 ,  0.58604286],
       [-0.56808398, -0.50784934, -0.86176938, ...,  0.90513041,
         0.41570668, -0.38506427],
       ...,
       [-0.23539182, -0.50784934,  1.17955762, ..., -1.82192738,
         0.32313459, -1.55879807],
       [-0.5113909 , -0.50784934, -0.71849348, ..., -0.48318992,
         0.34967446, -0.38956708],
       [-0.03148414, -0.50784934,  1.17955762, ..., -1.82192738,
        -1.57465677, -0.3745577 ]])

In [15]:
from keras import models
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(loss = 'mse', optimizer = 'rmsprop', metrics=['mae'])
    return model

In [18]:
k = 3
num_val_samples = len(train_data)//k

num_epochs = 200

all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples:(i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1)*num_val_samples]
    partial_train_data = np.concatenate([train_data[:i*num_val_samples],
                                        train_data[(i+1)*num_val_samples:]], axis = 0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                       train_targets[(i+1)*num_val_samples:]], axis = 0)
    
    model = build_model()
    model.fit(partial_train_data,partial_train_targets,
             epochs = num_epochs,batch_size=1,verbose=0)
    val_mse,val_mae = model.evaluate(val_data,val_targets,verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2


In [19]:
all_scores

[4.211258888244629, 3.368643045425415, 3.9292056560516357]

In [20]:
np.mean(all_scores)

3.836369196573893

In [22]:
k = 4
num_val_samples = len(train_data)//k

num_epochs = 500
all_mae_histories = []


for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples:(i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1)*num_val_samples]
    partial_train_data = np.concatenate([train_data[:i*num_val_samples],
                                        train_data[(i+1)*num_val_samples:]], axis = 0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                       train_targets[(i+1)*num_val_samples:]], axis = 0)
    
    model = build_model()
    history = model.fit(partial_train_data,partial_train_targets,
             epochs = num_epochs,batch_size=1,verbose=0, validation_data=(val_data,val_targets))
    mae_history = history.history['mean_absolute_error']
    all_mae_histories.append(mae_history)
    

processing fold # 0


KeyError: 'mean_absolute_error'